# Creating testing data and verifying using Z3

In [1]:
from z3 import *
import numpy as np
import csv
import pandas as pd
from collections import OrderedDict

In [2]:
#a0 represents a blank cell, i.e., no one marked it yet
a0=[1,0,0]

#a1 represents that marking of Player 1
a1=[0,1,0]

#a2 represents that marking of Player 2
a2=[0,0,1]

#a function to check if a given configuration already determines a winner or not
def check(array,index,element):
    dup = array.copy()
    dup[index]=element
    
    #check rows for a match(a match is possible iff all the 3 positions are marked by the same element)
    if(dup[0]==element and dup[1]==element and dup[2]==element):
        return 0
    if(dup[3]==element and dup[4]==element and dup[5]==element):
        return 0
    if(dup[6]==element and dup[7]==element and dup[8]==element):
        return 0
        
    #check column for a match(as defined)
    if(dup[0]==element and dup[3]==element and dup[6]==element):
        return 0
    if(dup[1]==element and dup[4]==element and dup[7]==element):
        return 0
    if(dup[2]==element and dup[5]==element and dup[8]==element):
        return 0
        
    #check diagonals for a match(as defined)
    if(dup[0]==element and dup[4]==element and dup[8]):
        return 0
    elif(dup[2]==element and dup[4]==element and dup[6]):
        return 0
    
    return 1
    
        

In [3]:
#Round 1 - 1 move made by each Player
p=a0*9
training_data=[p.copy()]
for i in range(9):
    p=[a0]*9
    p[i]=a1
    for j in range(9):
        dup = p.copy()
        if(dup[j]==a0):
            dup[j]=a2
            training_data.append(dup)
            
#Round 2 - 2 moves made by each Player
training_data2=[]
for k in (training_data[1:]):
    for i in range(9):
        dup = k.copy()
        if(dup[i]==a0):
            dup[i]=a1
            for j in range(9):
                dup2 = dup.copy()
                if(dup2[j]==a0):
                    dup2[j]=a2
                    training_data2.append(dup2)


#need to convert the inner lists to tuples so they are hashable
training_data2 = set(map(tuple,(np.array(i).reshape(27,) for i in training_data2))) 

training_data2 = list(np.array(i).reshape(9,3,).tolist() for i in training_data2)


#Round 3 - 3 moves made by each Player
training_data3=[]
for k in (training_data2):
    for i in range(9):
        dup = k.copy()
        if(dup[i]==a0 and check(dup,i,a1)):
            dup[i]=a1
            for j in range(9):
                dup2 = dup.copy()
                if(dup2[j]==a0 and check(dup2,j,a2)):
                    dup2[j]=a2
                    training_data3.append(dup2)
                    
#need to convert the inner lists to tuples so they are hashable
training_data3 = set(map(tuple,(np.array(i).reshape(27,) for i in training_data3)))

training_data3 = list(np.array(i).reshape(9,3,).tolist() for i in training_data3)

#training data for 2 and 3 Rounds(as defined)
training_data = training_data2 + training_data3
#print(len(training_data2))
#print(len(training_data3))

In [4]:
#a function to print the data in matrix form
def print_test(training_data):
    for j in range(9):
        ele = training_data[j]
        if(ele == a1):
            print(" a1 ", end=" ")
        if(ele == a2):
            print(" a2 ", end=" ")
        if(ele == a0):
            print(" - ", end=" ")
        if((j+1)%3 == 0):
            print("\n")
#for i in training_data[:40]:
#    print_test(i)
#    print("============")    

In [5]:
#generating all the possible positions that Player 1 can make in the next round
#Note: We assume Player 1 makes the first move and then Player 2 for any given round 
training_set=[]
for i in (training_data):
    for j in range(9):
        p=[0]*9
        if(i[j]==a0):
            p[j]=1
            training_set.append(i + [p])

"""c=0
for i in training_set[5800:]:
    print("=====",c,"======")
    print_test(i)
    print(i[-1])
    print("============")
    c+=1
"""

'c=0\nfor i in training_set[5800:]:\n    print("=====",c,"======")\n    print_test(i)\n    print(i[-1])\n    print("============")\n    c+=1\n'

### Some necessary Z3 functions 

In [6]:
#the Z3 list that will keep track of a given configuration and check its validity
Y = [ [ Int("x_%s_%s" % (i+1, j+1)) for j in range(3) ] for i in range(9) ]

#the Z3 list that will keep track of a next configuration according to the move by Player 1 and check its validity
X = [ [ Int("y_%s_%s" % (i+1, j+1)) for j in range(3) ] for i in range(9) ]

#if there is a winning move for Player 1

def Z3_Cell(X):
    #Assert that Each cell in the encoding has only boolean values that is 0 or 1 for next config
    C = [ [ Or(X[i][j]==1,X[i][j]==0) for j in range(3)] for i in range(9) ] #Each cell has either 0 or 1
    C1 = [And(C[i][0],C[i][1],C[i][2]) for i in range(9)]
    C2 = [And(C1[3*i],C1[3*i+1],C1[3*i+2]) for i in range(3)]
    return(And(C2[0],C2[1],C2[2]))

def Z3_OX(X):
    #Assert that Each cell is valid in the sense it can only hold the values a0 a1 a2 for next config
    A = [(X[i][0]+X[i][1]+X[i][2]==1) for i in range(9)] #Each position is either 001,010 or 100
    A1 = [And(A[3*i],A[3*i+1],A[3*i+2]) for i in range(3)]
    return(And(A1[0],A1[1],A1[2]))

def check_win(X):
    #If any of the columns is having a match for Player 1
    C = [And(X[i*3][1] == 1,X[i*3+1][1] == 1,X[i*3+2][1] == 1) for i in range(3)]
    Cols = Or(C[0],C[1],C[2])

    #If any of the rows is having a match for Player 1
    R = [And(X[i][1] == 1,X[i+3][1] == 1,X[i+6][1] == 1) for i in range(3)]
    Rows = Or(R[0],R[1],R[2])

    #If any of the Diagonals is having a match for Player 1
    Diags = Or(And(X[0][1] == 1, X[4][1] == 1, X[8][1] == 1),And(X[2][1] == 1, X[4][1] == 1, X[6][1] == 1))


    #A Winning_move for Player 1 is possible only if there is a possible match in any row or column or diagonal
    #in the next move of Player 1
    return(Or(Cols,Rows,Diags))

#A function that returns if the Z3 model has a satisfying assignment for the given configuration
#with the next move of Player 1 such that there is a blocking move of Player 2
def check_block(Y):
    #r is true iff the rows that corresponds to the index in which 
    #there is a winning move of Player 2
    r=[And(Y[i][0]+Y[i+3][0]+Y[i+6][0]==1,Y[i][2]+Y[i+3][2]+Y[i+6][2]==2) for i in range(3)]
    rows = Or(r[0],r[1],r[2])
    
    #c is true iff the columns that corresponds to the index in which the Player 1 has played its next move
    #but there is still a winning move of Player 2
    c=[And(Y[i*3][0]+Y[i*3+1][0]+Y[i*3+2][0]==1,Y[i*3][2]+Y[i*3+1][2]+Y[i*3+2][2]==2) for i in range(3)]
    cols = Or(c[0],c[1],c[2])
    
    #d1 is true iff the left diagonal that corresponds to the index in which the Player 1 has played its next move
    # but there is still a winning move of Player 2
    d1=And(Y[0][0]+Y[4][0]+Y[8][0]==1,Y[0][2]+Y[4][2]+Y[8][2]==2)
    
    #d2 is true iff the right diagonal that corresponds to the index in which the Player 1 has played its next move
    #but there is still a winning move of Player 2
    d2=And(Y[2][0]+Y[4][0]+Y[6][0]==1,Y[2][2]+Y[4][2]+Y[6][2]==2)
    
    #if any of the diagonals,rows or columns has a winning move by Player 2
    #w.r.t. the given index of Player 1 then it returns False
    return((Or(d1,d2,rows,cols)))

#A function that returns if the Z3 model has a satisfying assignment for the given configuration
#with the next move of Player 1 such that move blocks a winning move of Player 2
def Winning_move_P1(X):
    #r is true iff the rows that corresponds to the index in which the Player 1 has played its next move
    #but there is still a winning move of Player 2
    r=[And(X[i][0]+X[i+3][0]+X[i+6][0]==1,X[i][1]+X[i+3][1]+X[i+6][1]==2) for i in range(3)]
    rows = Or(r[0],r[1],r[2])
    
    #c is true iff the columns that corresponds to the index in which the Player 1 has played its next move
    #but there is still a winning move of Player 2
    c=[And(X[i*3][0]+X[i*3+1][0]+X[i*3+2][0]==1,X[i*3][1]+X[i*3+1][1]+X[i*3+2][1]==2) for i in range(3)]
    cols = Or(c[0],c[1],c[2])
    
    #d1 is true iff the left diagonal that corresponds to the index in which the Player 1 has played its next move
    # but there is still a winning move of Player 2
    d1=And(X[0][0]+X[4][0]+X[8][0]==1,X[0][1]+X[4][1]+X[8][1]==2)
    
    #d2 is true iff the right diagonal that corresponds to the index in which the Player 1 has played its next move
    #but there is still a winning move of Player 2
    d2=And(X[2][0]+X[4][0]+X[6][0]==1,X[2][1]+X[4][1]+X[6][1]==2)
    
    #if any of the diagonals,rows or columns has a winning move by Player 1
    # then it returns true
    return((Or(d1,d2,rows,cols)))


#the given config shouldn't have a player who has already won
def Not_won(X):
    r=[Or(X[i][1]+X[i+3][1]+X[i+6][1]==3,X[i][2]+X[i+3][2]+X[i+6][2]==3) for i in range(3)]
    rows = Or(r[0],r[1],r[2])

    c=[Or(X[i*3][1]+X[i*3+1][1]+X[i*3+2][1]==3,X[i*3][2]+X[i*3+1][2]+X[i*3+2][2]==3) for i in range(3)]
    cols = Or(c[0],c[1],c[2])

    d1=Or(X[0][1]+X[4][1]+X[8][1]==3,X[0][2]+X[4][2]+X[8][2]==3)

    d2=Or(X[2][1]+X[4][1]+X[6][1]==3,X[2][2]+X[4][2]+X[6][2]==3)

    return(Not(Or(d1,d2,rows,cols)))


### the code for generating good or bad for each valid config

In [7]:
#The encoding for Good and Bad
good = [1,0]
bad = [0,1]

count=0
#initialize the variable that would store the configurations along with whether they are good or bad
verified_training_set=[]
for instance in training_set:
    #"instance_with_P1_move" includes the given configuration along with the next move of Player 1
    instance_with_P1_move = instance.copy()
    instance_k = [ And(Y[i][0] == instance_with_P1_move[i][0],
                       Y[i][1] == instance_with_P1_move[i][1],
                       Y[i][2] == instance_with_P1_move[i][2])
               for i in range(9)]
    s1 = [And(instance_k[i],instance_k[i+3],instance_k[i+6]) for i in range(3)]
    instance_j  = And(s1[0],s1[1],s1[2])
    #index is the position of Player 1's next move
    index=0
    for i in range(9):
        if(instance[-1][i]==1):
            index=i
    instance_with_P1_move[index]=a1
    
    #Asserting the Configuration to the Z3 list of variables
    instance_c = [ And(X[i][0] == instance_with_P1_move[i][0],
                       X[i][1] == instance_with_P1_move[i][1],
                       X[i][2] == instance_with_P1_move[i][2])
               for i in range(9)]
    s1 = [And(instance_c[i],instance_c[i+3],instance_c[i+6]) for i in range(3)]
    instance_s  = And(s1[0],s1[1],s1[2])
    #instance_s is true iff the configuration can be asserted to the Z3 variables
    
    #s is the solver that gives the constraints of Cell, OX_ as defined above
    #along with the constraint that if there is a winning move for Player 1 it will always
    #opt for that move, if there's no winning move, then checks for a blocking move and 
    #if there is a blocking move, then it will opt for one of them,otherwise it can put any move
    s = Solver()
    s.add(If(Winning_move_P1(Y),check_win(X),If(check_block(Y),Not(check_block(X)),True)),
          instance_s,instance_j)
    
    #print_test(instance)
    #print(instance[-1])
    #print_test(instance_with_P1_move)
    
    if s.check() == unsat:
        #print("Bad")
        verified_training_set.append(instance+[[0,1]]) #bad
    else:
        #print("Good")
        verified_training_set.append(instance+[[1,0]]) #good
    count+=1
    #print("=========",count)

# The Neural Network

In [40]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras 

### Dividing the dataset into training and testing data

In [41]:
# Creating a dataframe 
df = pd.DataFrame(verified_training_set.copy()) 
  
# Creating a dataframe with 50% 
# values of original dataframe 
part_50 = df.sample(frac = 0.5) 
print(np.array(part_50).shape)
# Creating dataframe with  
# rest of the 50% values 
rest_part_50 = df.drop(part_50.index) 
print(np.array(rest_part_50).shape)

(3333, 11)
(3333, 11)


In [46]:
#reshaping the data and splitting into input and output data
x_train=np.hstack(np.delete(np.array(part_50),-1,axis=1))
x_train=np.hstack(x_train).reshape(len(part_50),36,1)
#print(x_train)
y=np.delete(np.array(part_50),slice(10),axis=1)
y_t=np.hstack(y)
y_train=np.hstack(y_t).reshape(len(part_50),-1)
#print(y_train)

x_test=np.hstack(np.delete(np.array(rest_part_50),-1,axis=1))
x_test=np.hstack(x_test).reshape(len(rest_part_50),36,1)
#print(x_test)
y1=np.delete(np.array(rest_part_50),slice(10),axis=1)
y_t1=np.hstack(y1)
y_test=np.hstack(y_t1).reshape(len(rest_part_50),-1)
#print(y_test)

### The Neural Network

In [47]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(36,1)),
    keras.layers.Dense(units=25, activation='relu'),
    keras.layers.Dense(units=25, activation='relu'),
    keras.layers.Dense(units=2, activation='softmax')
])
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_8 (Flatten)          (None, 36)                0         
_________________________________________________________________
dense_24 (Dense)             (None, 25)                925       
_________________________________________________________________
dense_25 (Dense)             (None, 25)                650       
_________________________________________________________________
dense_26 (Dense)             (None, 2)                 52        
Total params: 1,627
Trainable params: 1,627
Non-trainable params: 0
_________________________________________________________________


In [48]:
model.compile(optimizer='adam', 
              loss="categorical_crossentropy",
              metrics=['accuracy'])

history = model.fit(x_train, y_train,epochs=35)

Epoch 1/35
105/105 [==============================] - 0s 2ms/step - loss: 0.6750 - accuracy: 0.5755
Epoch 2/35
105/105 [==============================] - 0s 1ms/step - loss: 0.6554 - accuracy: 0.6118
Epoch 3/35
105/105 [==============================] - 0s 1ms/step - loss: 0.6431 - accuracy: 0.6283
Epoch 4/35
105/105 [==============================] - 0s 1ms/step - loss: 0.6299 - accuracy: 0.6409
Epoch 5/35
105/105 [==============================] - 0s 2ms/step - loss: 0.6152 - accuracy: 0.6583
Epoch 6/35
105/105 [==============================] - 0s 1ms/step - loss: 0.5992 - accuracy: 0.6760
Epoch 7/35
105/105 [==============================] - 0s 2ms/step - loss: 0.5871 - accuracy: 0.6817
Epoch 8/35
105/105 [==============================] - 0s 2ms/step - loss: 0.5761 - accuracy: 0.7033
Epoch 9/35
105/105 [==============================] - 0s 1ms/step - loss: 0.5645 - accuracy: 0.7063
Epoch 10/35
105/105 [==============================] - 0s 1ms/step - loss: 0.5543 - accuracy: 0.7144

In [49]:
model.evaluate(x_test,y_test)
output = model.predict(x_test)
for i in output:
        if(i[0]>=i[1]):
            i[0]=1
            i[1]=0
        else:
            i[0]=0
            i[1]=1

105/105 [==============================] - 0s 3ms/step - loss: 0.5064 - accuracy: 0.7552


### Extracting the weights and biases from the trained model

In [14]:
layers = []
for layer in model.layers: 
    layers.append(layer.get_weights())
    #print(layer.get_weights())
    #print("\n=============================================================\n")

first_layer_weights = layers[1][0].transpose()
first_layer_biases = layers[1][1].transpose()
second_layer_weights = layers[2][0].transpose()
second_layer_biases = layers[2][1].transpose()
third_layer_weights = layers[3][0].transpose()
third_layer_biases = layers[3][1].transpose()
weights = [first_layer_weights]+[second_layer_weights]+[third_layer_weights]
biases = [first_layer_biases]+[second_layer_biases]+[third_layer_biases]
print(weights[0].shape)
print(weights[1].shape)
print(weights[2].shape)

(20, 36)
(20, 20)
(2, 20)


# Testing the Trained Model using Z3

### Some important Z3 functions

In [15]:
def multiply_matrix(A, B):
    """
    Assuming valid matrix dimensions
    """
    result = [[0 for j in range(len(B[0]))] for i in range(len(A))]
    for i in range(len(A)): 
        for j in range(len(B[0])): 
            for k in range(len(B)):
                result[i][j] += A[i][k] * B[k][j]
    return result
#1-D matrix
def add_matrix(A, B):
    result = [0 for i in range(len(A))]
    for i in range(len(A)):  
        result[i] = A[i] + B[i]
    return result

def apply_relu(out):
    """
    Returns a new z3 list of variables on which relu is applied with name
    """
#     result = [z3.Real('out_'+ str(i)) for i in range(len(out))]
    curr = [[z3.If(i[0]>0, i[0], 0)] for i in out]
#     for i in range(len(result)):
#         sol.add(result[i] == curr[i][0])
    return curr

def feedforward_z3(weights, biases, x_z3,sol):
    """
    Return the output of the network for the input 'x' which is a list of z3 variables
    """

    layered_output = list()
    out = x_z3
    count = 1
    for b, w in zip(biases,weights):
        print("Working on layer:", count, "...")
        out0 = multiply_matrix(w, out)
        out1 = add_matrix(out0, b)
        if(count<len(biases)):
            out = apply_relu(out1)
        else:
            out=out1.copy()
        layered_output.append(out)
        count += 1
    return out

def get_solver(weights, biases): # for relu activation on each layer
    ## input shape can be obtained by the first weights matrix and output shape by the last weights matrix

    sol = z3.Solver()
    inp_size = len(weights[0][0])  ### number of columns in the first row of the first matrix
    out_size = len(weights[-1]) ### number of rows in the last weight matrix
    inp_z3 = [[Real('x_' + str(i))] for i in range(inp_size)]
    out_z3 = [[Real('y_' + str(i))] for i in range(out_size)]
    res = feedforward_z3(weights, biases, inp_z3, sol)
    for i in range(len(out_z3)):
        sol.add(res[i][0] == out_z3[i][0])
    return sol, inp_z3, out_z3
    ## let us calculate the output layer by layer using the weights and biases with relu activation

def get_solver1(weights, biases): # for relu activation on each layer
    ## input shape can be obtained by the first weights matrix and output shape by the last weights matrix

    sol = z3.Solver()
    inp_size = len(weights[0][0])  ### number of columns in the first row of the first matrix
    out_size = len(weights[-1]) ### number of rows in the last weight matrix
    inp_z3 = [[Real('p_' + str(i))] for i in range(inp_size)]
    out_z3 = [[Real('q_' + str(i))] for i in range(out_size)]
    res = feedforward_z3(weights, biases, inp_z3, sol)
    for i in range(len(out_z3)):
        sol.add(res[i][0] == out_z3[i][0])
    return sol, inp_z3, out_z3
    ## let us calculate the output layer by layer using the weights and biases with relu activation


## Question 3

### Getting the constraints for the output of the neural network

In [16]:
sol,inp_z3,out_z3 = get_solver(weights,biases)

Working on layer: 1 ...
Working on layer: 2 ...
Working on layer: 3 ...


### Using Z3 to get a configuration that satisfies the required properties mentioned in Q3...
##### Note: the output gives an assignment to all the Z3 variables if sat otherwise returns unsat
##### Note: x1 is the given config, y1 is the next move of Player1, z1 is the next config
##### x is the input to the network,i.e.,x1 and y is the output of the network

In [17]:
#Generate input where the configuration resulting from it
#has a possibility that Player2 has a winning move
#the Z3 list that will keep track of a given configuration and check its validity
X = [ [ Int("x1_%s_%s" % (i+1, j+1)) for j in range(3) ] for i in range(9) ]

#the Z3 list that keeps track of the next postion that Player 1 is going to mark
Y = [ Int("y1_%s" % (i+1)) for i in range(9)]

#next config
Z = [ [ Int("z1_%s_%s" % (i+1, j+1)) for j in range(3) ] for i in range(9)]


#assert that each cell in Y is having a single 1 and others are 0
#the cell having 1 corresponds to a tictactoe cell thats empty
B = [Or(Y[i]==0,Y[i]==1) for i in range(9)]
B_p = [And(B[i],B[i+3],B[i+6]) for i in range(3)]
Y_O1 = And(B_p[0],B_p[1],B_p[2])
B1 = [ Y[i]+Y[i+3]+Y[i+6] for i in range(3)]
Y_only1 = (B1[0]+B1[1]+B1[2]==1)
B3 = [If(Y[i]==1,X[i][0]==1,True) for i in range(9)]
B_p = [And(B3[i],B3[i+3],B3[i+6]) for i in range(3)]
Y_val_config = And(B_p[0],B_p[1],B_p[2])
val_Y = And(Y_only1, Y_val_config, Y_O1)

#2/3 moves by each Player
A1 = [(X[i][1]+X[i+3][1]+X[i+6][1]) for i in range(3)]
p1 = A1[0]+A1[1]+A1[2] 
A1 = [(X[3*i][2]+X[3*i+1][2]+X[3*i+2][2]) for i in range(3)]
p2 = A1[0]+A1[1]+A1[2]
Valid_config = Or(And(p1==2,p2==2),And(p1==3,p2==3))

#the condition in Question 3
#NEXT config after player 1's move
instance_k = [ If(Y[i]==1,And(Z[i][0] == 0,
                       Z[i][1] == 1,
                       Z[i][2] == 0),
                  And(Z[i][0] == X[i][0],
                       Z[i][1] == X[i][1],
                       Z[i][2] == X[i][2]))
               for i in range(9)]
s1 = [And(instance_k[i],instance_k[i+3],instance_k[i+6]) for i in range(3)]
instance_next  = And(s1[0],s1[1],s1[2])

sol.add(Z3_Cell(X),val_Y,Z3_OX(X),Valid_config,Not_won(X),instance_next,If(check_win(Z),False,check_block(Z)))
count=0
for i in range(9):
    for j in range(3):
        sol.add(inp_z3[count][0] == X[i][j])
        count+=1
for i in range(9):
    sol.add(inp_z3[27+i][0] == Y[i])
    
sol.add(out_z3[0][0] > out_z3[1][0])

if sol.check()==sat:
    print(sol.model())
else:
    print("unsat: There is no input satisfying the constraint")

[z1_7_3 = 0,
 z1_8_1 = 0,
 y1_3 = 0,
 y1_9 = 0,
 z1_1_2 = 0,
 z1_3_1 = 0,
 z1_5_2 = 0,
 x1_3_3 = 1,
 z1_3_2 = 0,
 z1_4_1 = 0,
 x1_2_2 = 0,
 z1_7_2 = 1,
 z1_7_1 = 0,
 z1_9_1 = 0,
 x1_7_3 = 0,
 x1_5_2 = 0,
 z1_3_3 = 1,
 y1_8 = 0,
 z1_5_3 = 0,
 z1_2_3 = 1,
 z1_1_3 = 0,
 z1_6_3 = 0,
 x1_6_2 = 0,
 x1_8_3 = 1,
 z1_6_2 = 1,
 y1_6 = 1,
 z1_4_3 = 0,
 x1_3_2 = 0,
 x1_9_2 = 1,
 x1_8_2 = 0,
 y1_7 = 0,
 z1_6_1 = 0,
 z1_2_2 = 0,
 x1_6_3 = 0,
 z1_8_3 = 1,
 z1_9_3 = 0,
 x1_9_3 = 0,
 y1_5 = 0,
 x1_1_2 = 0,
 x1_4_2 = 1,
 z1_2_1 = 0,
 x1_5_3 = 0,
 x1_4_3 = 0,
 z1_5_1 = 1,
 x1_2_3 = 1,
 y1_2 = 0,
 z1_8_2 = 0,
 z1_9_2 = 1,
 x1_7_2 = 1,
 y1_4 = 0,
 z1_1_1 = 1,
 z1_4_2 = 1,
 y_1 = 1891384448804422010640252587897961133914024620992241949/10000000000000000000000000000000000000000000000000000000,
 y_0 = 3547266444048930420895341148338421864316019749671210101/10000000000000000000000000000000000000000000000000000000,
 x_35 = 0,
 x_34 = 0,
 x_33 = 0,
 x_32 = 1,
 x_31 = 0,
 x_30 = 0,
 x_29 = 0,
 x_28 = 0,
 x_27 = 0,

## Question 4

### Getting the constraints for the output of the neural network

In [18]:
sol,inp_z3,out_z3 = get_solver1(weights,biases)

Working on layer: 1 ...
Working on layer: 2 ...
Working on layer: 3 ...


### Using Z3 to get a configuration that satisfies the required properties mentioned in Q4...
##### Note: the output gives an assignment to all the Z3 variables if sat otherwise returns unsat
##### Note: x2 is the given config, y2 is the next move of Player1, z2 is the next config
##### p is the input to the network,i.e.,x2 and q is the output of the network 

In [19]:
#Generate input where the configuration resulting from it
#has a possibility that Player2 has a winning move
#the Z3 list that will keep track of a given configuration and check its validity
X = [ [ Int("x2_%s_%s" % (i+1, j+1)) for j in range(3) ] for i in range(9) ]

#the Z3 list that keeps track of the next postion that Player 1 is going to mark
Y = [ Int("y2_%s" % (i+1)) for i in range(9)]

#next config
Z = [ [ Int("z2_%s_%s" % (i+1, j+1)) for j in range(3) ] for i in range(9)]


#assert that each cell in Y is having a single 1 and others are 0
#the cell having 1 corresponds to a tictactoe cell thats empty
B = [Or(Y[i]==0,Y[i]==1) for i in range(9)]
B_p = [And(B[i],B[i+3],B[i+6]) for i in range(3)]
Y_O1 = And(B_p[0],B_p[1],B_p[2])
B1 = [ Y[i]+Y[i+3]+Y[i+6] for i in range(3)]
Y_only1 = (B1[0]+B1[1]+B1[2]==1)
B3 = [If(Y[i]==1,X[i][0]==1,True) for i in range(9)]
B_p = [And(B3[i],B3[i+3],B3[i+6]) for i in range(3)]
Y_val_config = And(B_p[0],B_p[1],B_p[2])
val_Y = And(Y_only1, Y_val_config, Y_O1)

#2/3 moves by each Player
A1 = [(X[i][1]+X[i+3][1]+X[i+6][1]) for i in range(3)]
p1 = A1[0]+A1[1]+A1[2] 
A1 = [(X[3*i][2]+X[3*i+1][2]+X[3*i+2][2]) for i in range(3)]
p2 = A1[0]+A1[1]+A1[2]
Valid_config = Or(And(p1==2,p2==2),And(p1==3,p2==3))

#the condition in Question 3
#NEXT config after player 1's move
instance_k = [ If(Y[i]==1,And(Z[i][0] == 0,
                       Z[i][1] == 1,
                       Z[i][2] == 0),
                  And(Z[i][0] == X[i][0],
                       Z[i][1] == X[i][1],
                       Z[i][2] == X[i][2]))
               for i in range(9)]
s1 = [And(instance_k[i],instance_k[i+3],instance_k[i+6]) for i in range(3)]
instance_next  = And(s1[0],s1[1],s1[2])


sol.add(Z3_Cell(X),val_Y,Z3_OX(X),Valid_config,Not_won(X),instance_next,If(check_win(Z),False,check_block(Z)))
sol.add(X[4][1]==1)
sol.add(Or(X[0][1]==1,X[2][1]==1,X[6][1]==1,X[8][1]==1))

count=0
for i in range(9):
    for j in range(3):
        sol.add(inp_z3[count][0] == X[i][j])
        count+=1
for i in range(9):
    sol.add(inp_z3[27+i][0] == Y[i])
    
sol.add(out_z3[0][0] > out_z3[1][0])

if sol.check()==sat:
    print(sol.model())
else:
    print("unsat: There is no input satisfying the constraint")

[z2_8_3 = 0,
 x2_7_2 = 0,
 x2_9_2 = 0,
 x2_2_2 = 0,
 y2_3 = 0,
 z2_6_2 = 0,
 x2_6_2 = 0,
 y2_6 = 0,
 x2_6_3 = 1,
 z2_2_3 = 0,
 z2_3_3 = 0,
 z2_5_1 = 0,
 z2_7_2 = 0,
 z2_2_1 = 1,
 z2_1_3 = 1,
 z2_3_1 = 0,
 z2_9_1 = 0,
 y2_4 = 0,
 z2_9_2 = 1,
 x2_3_3 = 0,
 z2_7_1 = 1,
 z2_8_2 = 1,
 z2_4_1 = 0,
 z2_4_2 = 0,
 y2_9 = 1,
 z2_5_3 = 0,
 z2_3_2 = 1,
 z2_6_1 = 0,
 x2_5_3 = 0,
 x2_9_3 = 0,
 z2_6_3 = 1,
 x2_8_2 = 1,
 x2_2_3 = 0,
 x2_1_3 = 1,
 x2_4_3 = 1,
 z2_4_3 = 1,
 x2_4_2 = 0,
 x2_1_2 = 0,
 x2_7_3 = 0,
 y2_2 = 0,
 z2_9_3 = 0,
 x2_8_3 = 0,
 x2_3_2 = 1,
 y2_8 = 0,
 z2_7_3 = 0,
 z2_2_2 = 0,
 y2_7 = 0,
 z2_8_1 = 0,
 z2_1_2 = 0,
 z2_1_1 = 0,
 y2_5 = 0,
 z2_5_2 = 1,
 p_13 = 1,
 x2_5_2 = 1,
 q_1 = -2511246911814881544166186461629603898599106799355449/10000000000000000000000000000000000000000000000000000,
 q_0 = 81832487953345473470515000191449727065918548128522337/250000000000000000000000000000000000000000000000000000,
 p_35 = 1,
 p_34 = 0,
 p_33 = 0,
 p_32 = 0,
 p_31 = 0,
 p_30 = 0,
 p_29 = 0,
 p_28 